In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import json
import re
import time
import os
import re

domain = 'https://www.8book.com'

In [2]:
# get_book_data(url)
# Inputs: book's url <str>
# Ouputs: chapters list <list>, book's name <str>, book's type <str>, book's chapters' count <int>

def get_book_data(url):
    chapter_list = []
    name = ''
    type = ''
    count = 0

    html = requests.get(url)
    html.encoding = 'utf8'
    soup = BeautifulSoup(html.text, 'html.parser')

    for i in soup.find_all('meta'):
        if i.get('name') == 'cat': 
            type = i.get('content')
        elif i.get('name') == 'name': 
            name = i.get('content')
        elif i.get('name') == 'count':
            count = int(re.sub(r'\D','',i.get('content')))

    chapters = soup.find_all(id='chapters')[0]
    for i in chapters.findAll('a'):
        chapter_list.append(domain+i.get('href'))
    
    return chapter_list, name, type, count

# get_book_data('https://www.8book.com/novelbooks/104934')[0][1]

In [3]:
# get_article(url_)
# Inputs: chapter's url <str>
# Ouputs: article text <list>

def get_article(url_):
    # url_ = 'https://www.8book.com/read/104934/?20740_2'
    url = f'http://localhost:8050/render.html?url={url_}&timeout=10&wait=0.5'
    html = requests.get(url)
    html.encoding = 'utf8'
    soup = BeautifulSoup(html.text, 'html.parser')

    texts = ''
    for i in soup.find_all(name='div',attrs={'class':'text'}):
        texts += i.text
    texts = texts.replace('\u3000','').replace('\n','')
    texts = re.sub('[a-zA-Z0-9]','',texts)
    return texts

# get_article('https://www.8book.com/read/104934/?20740_2')

In [4]:
book_list = []

a = 1 
while True:

    url = f'https://www.8book.com/booklists/11/{a}.html'
    html = requests.get(url)
    html.encoding = 'utf8'

    try:
        html = html.text.split('<!-- all list -->')[1]
        soup = BeautifulSoup(html, 'html.parser')

        for i in list(set([i.get('href') for i in soup.find_all('a') if i.get('class') != None])):
            book_list.append(domain+i)

        a += 1
        time.sleep(0.2)
    except:
        break

In [5]:
# utils

path = 'atricles/literature'
if not os.path.isdir(path):
    os.makedirs(path)

for book in tqdm(book_list):
    # get book's info
    id = book.split('/')[-1]
    url = 'https://www.8book.com/novelbooks/'+id
    cp_list, book_name, book_type, book_count = get_book_data(url) 
    info = {'name':book_name,'type':book_type,'count':book_count}

    # if book not exsists, creat file and update the info.
    if  os.path.isdir(path+'/'+id):
        pass 
    else:
        json_ = {'info':info,'chapters':{}}
        with open(path+'/'+id+'.json', "w") as f:
            json.dump(json_, f)

    # check if chapter is null, if it's null, update it.
    with open(path+'/'+id+'.json', "r") as f:
        json_ = json.load(f)
    charpter = json_['chapters']
    for i in range(1,len(cp_list)+1):
        if str(i) in list(charpter.keys()):
            pass
        else:
            try:
                charpter[i] = get_article(cp_list[i-1])
            except:
                pass
        json_['chapters'] = charpter
    
    # save to the file
    with open(path+'/'+id+'.json', "w") as f:
        json.dump(json_, f)
    


  0%|          | 1/432 [01:39<11:56:57, 99.81s/it]

In [ ]:
# check state
with open(path+'/'+id+'.json', "r") as f:
    json_ = json.load(f)
print((json_))